# Getting seasonal forecast data

In [1]:
options(java.parameters = "-Xmx100000m")
library(transformeR)
library(loadeR.ECOMS)
library(loadeR)
library(visualizeR)
library(convertR)
library(drought4R)
library(downscaleR)

install.packages("rlist")
library("rlist")

transformeR version 1.5.1 (2019-07-13) is loaded
Get the latest stable version (1.6.0) using <devtools::install_github('SantanderMetGroup/transformeR')>
Please see 'citation("transformeR")' to cite this package.
Loading required package: loadeR
Loading required package: rJava
Loading required package: loadeR.java
Java version 1.8x amd64 by Azul Systems, Inc. detected
NetCDF Java Library v4.6.0-SNAPSHOT (23 Apr 2015) loaded and ready
loadeR version 1.4.15 (2019-07-30) is loaded
Please use 'citation("loadeR")' to cite this package.
loadeR.ECOMS version 1.4.6 (2018-10-20) is loaded
Please use 'citation("loadeR.ECOMS")' to cite this package.
Warning message:
“no DISPLAY variable so Tk is not available”visualizeR version 1.4.0 (2019-07-09) is loaded
Get the latest stable version (1.5.0) using <devtools::install_github('SantanderMetGroup/visualizeR')>
Please see 'citation("visualizeR")' to cite this package.
Loading required package: udunits2
udunits system database read from /opt/conda/lib/

In [2]:
# Output path where the data will be saved (change to your local path).
dir.data <- './data/' 
dir.Rdata <- './Rdata/'


#Defining the geographical domain
lonLim <- c(8, 13) 
latLim <- c(57, 62)

# Define the period and the season
years <- 1979:2019
season <- 1:12 #Full year

# Location of the lake
lake <- list(x= 10.7438771059,  y= 59.4392814052 )
lakename <- 'Vansjo'
    

# Define the dataset 
dataset <- "System4_seasonal_15"

loadeR::loginUDG(username='WATExR',password="1234567890")

# Check available variables in the dataset 
di <- dataInventory("http://www.meteo.unican.es/tds5/dodsC/system4/System4_Seasonal_15Members.ncml")
names(di)

[2019-10-17 08:55:05] Setting credentials...
[2019-10-17 08:55:06] Success!
Go to <http://www.meteo.unican.es/udg-tap/home> for details on your authorized groups and datasets
[2019-10-17 08:55:07] Doing inventory ...
[2019-10-17 08:55:15] Retrieving info for 'z1000mb' (45 vars remaining)
[2019-10-17 08:55:17] Retrieving info for 'z700mb' (44 vars remaining)
[2019-10-17 08:55:18] Retrieving info for 'z850mb' (43 vars remaining)
[2019-10-17 08:55:18] Retrieving info for 'z500mb' (42 vars remaining)
[2019-10-17 08:55:19] Retrieving info for 'z300mb' (41 vars remaining)
[2019-10-17 08:55:20] Retrieving info for 'z200mb' (40 vars remaining)
[2019-10-17 08:55:20] Retrieving info for 't1000mb' (39 vars remaining)
[2019-10-17 08:55:21] Retrieving info for 't700mb' (38 vars remaining)
[2019-10-17 08:55:21] Retrieving info for 't850mb' (37 vars remaining)
[2019-10-17 08:55:22] Retrieving info for 't500mb' (36 vars remaining)
[2019-10-17 08:55:23] Retrieving info for 't300mb' (35 vars remaining)


[1] "z1000mb"  "z700mb"   "z850mb"   "z500mb"   "z300mb"   "z200mb"  
 [7] "t1000mb"  "t700mb"   "t850mb"   "t500mb"   "t300mb"   "t200mb"  
[13] "u1000mb"  "u700mb"   "u850mb"   "u500mb"   "u300mb"   "u200mb"  
[19] "v1000mb"  "v700mb"   "v850mb"   "v500mb"   "v300mb"   "v200mb"  
[25] "q1000mb"  "q700mb"   "q850mb"   "q500mb"   "q300mb"   "q200mb"  
[31] "zsfc"     "ssrd"     "strd"     "tcc"      "u10m"     "v10m"    
[37] "dpt2m"    "t2m"      "mean2t24" "sst"      "mx2t24"   "mn2t24"  
[43] "sd"       "sf"       "tp"       "mslp"

In [3]:
# Path to the observational data (change to your local path).
dir.Rdata.obs <- "./Rdata/PIK_Obs-EWEMBI_1_2_3_4_5_6_7_8_9_10_11_12_uas_vas_ps_tas_pr_rsds_rlds_hurs_cc.rda"
obs.data <- get(load(dir.Rdata.obs))

In [4]:
# Define the variables to be loaded (the same as in the observational data, 
# except clould cover (cc) and evapotranspiration (petH))
sapply(obs.data, function(x) getVarNames(x)) # to check the variables in the observational data.
variables <- c("uas", "vas", "ps", "tas", "pr", "rsds", "rlds", "hurs")

# Define daily aggregation function for each variable selected
aggr.fun <- c("mean", "mean", "mean", "mean", "sum", "mean", "mean", "mean")

# Define the members
mem <- 1:15
# Define the lead month
lead.month <- 0
# Define period and season
years <- 1981:2000

uas    vas     ps    tas     pr   rsds   rlds   hurs     cc 
 "uas"  "vas"   "ps"  "tas"   "pr" "rsds" "rlds" "hurs"   "cc"

In [5]:
########## DATA LOADING AND TRANSFORMATION ----------------------------------------------------------

# Load seasonal forecast data (System4 or CFS) with function loadECOMS
# Data is loaded in a loop (función lapply) to load all variables in a single code line.
# A list of grids is obtained, each slot in the list corresponds to a variable


In [6]:
season <- c(2,3,4,5) # Spring
rm(data)
# Load seasonal forecast data (System4 or CFS) with function loadECOMS
# Data is loaded in a loop (función lapply) to load all variables in a single code line.
# A list of grids is obtained, each slot in the list corresponds to a variable
data.prelim <- lapply(1:length(variables), function(x) loadECOMS(dataset, var = variables[x], years = years, 
                                                          members = mem, leadMonth = lead.month,
                                                          lonLim = lonLim, latLim = latLim, season = season, 
                                                          time = "DD", aggr.d = aggr.fun[x]))
names(data.prelim) <- variables

# Bilinear interpolation of the data to the location of the lake
data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
                                                          method = "bilinear", 
                                                          bilin.method = "akima"))

# Convert pressure units to millibars with function udConvertGrid from package convertR.
data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

# Compute cloud cover with function rad2cc
clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
clt$Variable$varName <- "cc"

# Put all variables together
data <- c(data.interp, "cc" = list(clt))
list.save(data, "era_system4_spring.rds")

[2019-10-17 08:55:39] Defining harmonization parameters for variable "uas"
[2019-10-17 08:55:39] Opening dataset...
[2019-10-17 08:55:41] The dataset was successfuly opened
[2019-10-17 08:55:41] Defining geo-location parameters
NOTE: 'leadMonth = 0' selected
[2019-10-17 08:55:41] Defining initialization time parameters
NOTE: Daily aggregation will be computed from 6-hourly data
[2019-10-17 08:55:43] Retrieving data subset ...
[2019-10-17 09:09:24] Done
[2019-10-17 09:09:24] Defining harmonization parameters for variable "vas"
[2019-10-17 09:09:24] Opening dataset...
[2019-10-17 09:09:26] The dataset was successfuly opened
[2019-10-17 09:09:26] Defining geo-location parameters
NOTE: 'leadMonth = 0' selected
[2019-10-17 09:09:26] Defining initialization time parameters
NOTE: Daily aggregation will be computed from 6-hourly data
[2019-10-17 09:09:30] Retrieving data subset ...
[2019-10-17 09:23:13] Done
NOTE: The requested variable is not originally stored in the requested database
It wil

In [7]:
season <- c(5,6,7,8) # Summer
rm(data)
# Load seasonal forecast data (System4 or CFS) with function loadECOMS
# Data is loaded in a loop (función lapply) to load all variables in a single code line.
# A list of grids is obtained, each slot in the list corresponds to a variable
data.prelim <- lapply(1:length(variables), function(x) loadECOMS(dataset, var = variables[x], years = years, 
                                                          members = mem, leadMonth = lead.month,
                                                          lonLim = lonLim, latLim = latLim, season = season, 
                                                          time = "DD", aggr.d = aggr.fun[x]))
names(data.prelim) <- variables

# Bilinear interpolation of the data to the location of the lake
data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
                                                          method = "bilinear", 
                                                          bilin.method = "akima"))

# Convert pressure units to millibars with function udConvertGrid from package convertR.
data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

# Compute cloud cover with function rad2cc
clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
clt$Variable$varName <- "cc"

# Put all variables together
data <- c(data.interp, "cc" = list(clt))
list.save(data, "era_system4_summer.rds")
                      
                      

[2019-10-17 10:41:07] Defining harmonization parameters for variable "uas"
[2019-10-17 10:41:08] Opening dataset...
[2019-10-17 10:41:10] The dataset was successfuly opened
[2019-10-17 10:41:10] Defining geo-location parameters
NOTE: 'leadMonth = 0' selected
[2019-10-17 10:41:10] Defining initialization time parameters
NOTE: Daily aggregation will be computed from 6-hourly data
[2019-10-17 10:41:13] Retrieving data subset ...
[2019-10-17 10:54:07] Done
[2019-10-17 10:54:07] Defining harmonization parameters for variable "vas"
[2019-10-17 10:54:07] Opening dataset...
[2019-10-17 10:54:09] The dataset was successfuly opened
[2019-10-17 10:54:09] Defining geo-location parameters
NOTE: 'leadMonth = 0' selected
[2019-10-17 10:54:09] Defining initialization time parameters
NOTE: Daily aggregation will be computed from 6-hourly data
[2019-10-17 10:54:13] Retrieving data subset ...
[2019-10-17 11:06:29] Done
NOTE: The requested variable is not originally stored in the requested database
It wil

In [ ]:
season <- c(8,9,10,11) # Fall
rm(data)
# Load seasonal forecast data (System4 or CFS) with function loadECOMS
# Data is loaded in a loop (función lapply) to load all variables in a single code line.
# A list of grids is obtained, each slot in the list corresponds to a variable
data.prelim <- lapply(1:length(variables), function(x) loadECOMS(dataset, var = variables[x], years = years, 
                                                          members = mem, leadMonth = lead.month,
                                                          lonLim = lonLim, latLim = latLim, season = season, 
                                                          time = "DD", aggr.d = aggr.fun[x]))
names(data.prelim) <- variables

# Bilinear interpolation of the data to the location of the lake
data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
                                                          method = "bilinear", 
                                                          bilin.method = "akima"))

# Convert pressure units to millibars with function udConvertGrid from package convertR.
data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

# Compute cloud cover with function rad2cc
clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
clt$Variable$varName <- "cc"

# Put all variables together
data <- c(data.interp, "cc" = list(clt))
list.save(data, "era_system4_fall.rds")

[2019-10-17 12:09:33] Defining harmonization parameters for variable "uas"
[2019-10-17 12:09:33] Opening dataset...
[2019-10-17 12:09:34] The dataset was successfuly opened
[2019-10-17 12:09:34] Defining geo-location parameters
NOTE: 'leadMonth = 0' selected
[2019-10-17 12:09:34] Defining initialization time parameters
NOTE: Daily aggregation will be computed from 6-hourly data
[2019-10-17 12:09:35] Retrieving data subset ...
[2019-10-17 12:19:48] Done
[2019-10-17 12:19:48] Defining harmonization parameters for variable "vas"
[2019-10-17 12:19:48] Opening dataset...
[2019-10-17 12:19:49] The dataset was successfuly opened
[2019-10-17 12:19:49] Defining geo-location parameters
NOTE: 'leadMonth = 0' selected
[2019-10-17 12:19:49] Defining initialization time parameters
NOTE: Daily aggregation will be computed from 6-hourly data
[2019-10-17 12:19:50] Retrieving data subset ...
[2019-10-17 12:29:53] Done
NOTE: The requested variable is not originally stored in the requested database
It wil

In [ ]:
season <- c(11,12,1,2) # Winter
rm(data)

data.prelim <- lapply(1:length(variables), function(x) loadECOMS(dataset, var = variables[x], years = years, 
                                                          members = mem, leadMonth = lead.month,
                                                          lonLim = lonLim, latLim = latLim, season = season, 
                                                          time = "DD", aggr.d = aggr.fun[x]))
names(data.prelim) <- variables

# Bilinear interpolation of the data to the location of the lake
data.interp <- lapply(data.prelim, function(x) interpGrid(x, new.coordinates = lake, 
                                                          method = "bilinear", 
                                                          bilin.method = "akima"))

# Convert pressure units to millibars with function udConvertGrid from package convertR.
data.interp$ps <- udConvertGrid(data.interp$ps, new.units = "millibars")

# Compute cloud cover with function rad2cc
clt <- rad2cc(rsds = data.interp$rsds, rlds = data.interp$rlds)
clt$Variable$varName <- "cc"

# Put all variables together
data <- c(data.interp, "cc" = list(clt))
list.save(data, "era_system4_winter.rds")